In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Loading the data and checking shape,as shown below there are 33K+ rows and 8 columns.

In [ ]:
data=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
data.head()

In [ ]:
data.shape

Checking the null values,as shown below sex,age_approx,anatom_site_general_challenge have null values.

In [ ]:
data.isnull().sum()

Using info(),to see the type of features.

In [ ]:
data.info()

# Filling null values

In [ ]:
data['sex'].fillna(data['sex'].mode()[0],inplace=True)
data['sex']=data['sex'].map({'male': 1, 'female': 0})

In [ ]:
data['age_approx'].fillna(data['age_approx'].median(),inplace=True)


In [ ]:
data['anatom_site_general_challenge'].fillna(data['anatom_site_general_challenge'].mode()[0],inplace=True)

In [ ]:
data.isnull().sum().sum()

As shown above,there are no missing values in th data.

# Target feature

Firstly checking the count of benign and malignant rows,and visualising them using countplot and pie chart.

In [ ]:
data['target'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.countplot(data['target'])

As shown above the data is completely imbalanced.

In [ ]:
plt.pie(data['target'].value_counts(),labels=[0,1],autopct='%1.2f%%')

# Sex feature

Here 1 means Male and 0 means Female. 

In [ ]:
sns.countplot(data['sex'])

As shown above, there are more males than females.

In [ ]:
plt.pie(data['sex'].value_counts(),labels=['male','female'],autopct='%1.2f%%')

Male percentage is 51.76 and Female ones is 48.24%.

Now visualising sex with target feature.

In [ ]:
sns.countplot(data['sex'],hue='target',data=data)

As shown malignant count is more in males than female.

# Age_approx feature

First finding the average age by sex,as shown below avg age in male is 50 and those of females is 47,by this we can conclude that females are effecting by this disease in their earlier age than males.

In [ ]:
data.groupby(['sex'])['age_approx'].mean()

In [ ]:
sns.kdeplot(data.loc[data['sex'] ==1, 'age_approx'], label = 'Male',shade=True)
sns.kdeplot(data.loc[data['sex'] ==0, 'age_approx'], label = 'Female',shade=True)

Now finding avg age by target,as shown below people whos age is 58+(elders) are malignant.

In [ ]:
data.groupby(['target'])['age_approx'].mean()

In [ ]:
sns.kdeplot(data.loc[data['target'] ==1, 'age_approx'], label = 'Malignant',shade=True)
sns.kdeplot(data.loc[data['target'] ==0, 'age_approx'], label = 'Bening',shade=True)

Now visualising age_approx feature using histogram,there are more people whose age are ranging between 43 and 62.

In [ ]:
data['age_approx'].hist(color='g')

Now visualising the distribution of target with respect to age using violin plot.

In [ ]:
sns.violinplot(x='target',y='age_approx',data=data)

Now using barchart,we are plotting sex with age by target,as shown below in both male and female,if the age is above 50 we can consider it as malignant.

In [ ]:
sns.barplot(x='sex',y='age_approx',hue='target',data=data)

# anatom_site_general_challenge feature

This feature indicates location of imaged site i.e location of symptoms on the body.As shown below most affected body part is torso(the trunk of the human body).

In [ ]:
data['anatom_site_general_challenge'].value_counts()

In [ ]:
data['anatom_site_general_challenge'].value_counts().plot(kind='bar')

Now visualising the effected part by sex.In male torso part is most effected and in remaining almost every part is heavyly effected by female.

In [ ]:
sns.countplot(x='anatom_site_general_challenge',hue='sex',data=data)
plt.xticks(rotation='vertical')

# Diagnosis feature

This feature indicates detailed diagnosis information.As shown below,in most of the cases it is unknown.

In [ ]:
data['diagnosis'].value_counts()

# Image data

First randomly visualising 9 images.

In [ ]:
IMAGE_PATH = "../input/siim-isic-melanoma-classification/"
from PIL import Image
images = data['image_name'].values
random_images = [np.random.choice(images+'.jpg') for i in range(9)]
img_dir = IMAGE_PATH+'/jpeg/train'
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = Image.open(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

Now we see random image by bening and malignant.

# Benign images

In [ ]:
benign =data[data['benign_malignant']=='benign']
malignant =data[data['benign_malignant']=='malignant']
images = benign['image_name'].values
random_images = [np.random.choice(images+'.jpg') for i in range(9)]
img_dir = IMAGE_PATH+'/jpeg/train'
plt.figure(figsize=(10,8))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

# Malignant images

In [ ]:
images = malignant['image_name'].values
random_images = [np.random.choice(images+'.jpg') for i in range(6)]
img_dir = IMAGE_PATH+'/jpeg/train'
plt.figure(figsize=(10,8))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

Now we will visualise images by sex.

# Male images

In [ ]:
Male =data[data['sex']==1]
Female =data[data['sex']==0]
images = Male['image_name'].values
random_images = [np.random.choice(images+'.jpg') for i in range(6)]
img_dir = IMAGE_PATH+'/jpeg/train'
plt.figure(figsize=(10,8))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

# Female images

In [ ]:
images = Female['image_name'].values
random_images = [np.random.choice(images+'.jpg') for i in range(6)]
img_dir = IMAGE_PATH+'/jpeg/train'
plt.figure(figsize=(10,8))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

# Please upvote if you like,any suggestions and mistakes put it in comments,Thank you.